In [2]:
import json
import itertools
import pickle
import hickle 
import gzip
import operator
import os
import sys
from time import time
import pprint as pp
import collections
import ConfigParser

import numpy as np
import pandas as pd

import twitter

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.externals import joblib


# bokeh
import bokeh.plotting as bkplt


# import requirments 
from IPython.display import Image
from IPython.display import display
import matplotlib.pyplot as plt
import json
import rpy2
%load_ext rpy2.ipython
%R require("ggplot2")
% matplotlib inline
from ggplot import *
randn = np.random.randn

/Users/blehman/anaconda/lib/python2.7/site-packages/rpy2/robjects/robject.py:5: UserWarning: Loading required package: stats

  rpy2.rinterface.initr()
/Users/blehman/anaconda/lib/python2.7/site-packages/rpy2/robjects/robject.py:5: UserWarning: Loading required package: DBI

  rpy2.rinterface.initr()
/Users/blehman/anaconda/lib/python2.7/site-packages/rpy2/robjects/robject.py:5: UserWarning: Loading required package: rJava

  rpy2.rinterface.initr()
/Users/blehman/anaconda/lib/python2.7/site-packages/rpy2/robjects/robject.py:5: UserWarning: Error in .jcall(drv@jdrv, "Ljava/sql/Connection;", "connect", as.character(url)[1],  : 
  java.sql.SQLException: [Vertica][VJDBC](100176) Failed to connect to host localhost on port 5434. Reason: Connection refused

  rpy2.rinterface.initr()


### Data
Let's grab some json records from Twitter's public api. 

We'll use python-twitter. 
<pre>
$ pip install python-twitter
$ pydoc twitter.Api
</pre>

Build an app [https://apps.twitter.com/](https://apps.twitter.com/).  

Then use the app info in the `config.cfg` file.
    

In [88]:
# read the config file.
config = ConfigParser.RawConfigParser()
config.read('myconfig.cfg')

# provided w/ Twitter app. See https://apps.twitter.com/
token = config.get('oauth','token')
token_secret = config.get('oauth','token_secret')
con_key = config.get('oauth','con_key')
con_secret_key = config.get('oauth','con_secret_key')

# setup 
api = twitter.Api(
    consumer_key=con_key
    , consumer_secret=con_secret_key
    , access_token_key = token
    , access_token_secret = token_secret)

# test creds
print "@{}".format(api.VerifyCredentials().GetScreenName())



BrianLehman


In [117]:
# run query
results = api.GetSearch(term = 'golden retriever'
                        , count = 100
                        , include_entities=True)
counter = 0
total_tweets = 5000
tweets = []
while counter <= total_tweets:
    if counter == 1:
        new_results = api.GetSearch(term = 'golden retriever'
                                    , count = 100
                                    , max_id = results[-1].GetId()
                                    , include_entities=True)
    else:
        new_results = api.GetSearch(term = 'golden retriever'
                                    , count = 100
                                    , max_id = new_results[-1].GetId()
                                    , include_entities=True)
    counter += len(new_results)
    tweets.extend(new_results)
#for i,item in enumerate(tweets):
#    print i,' ',item.GetText()


In [118]:
tweet_text = [tweet.GetText() for tweet in tweets]
len(tweet_text)

5081

### Build Topic Model
Three steps:  
1.  Set up a training and test set.
2.  Set up a vecterizer.
3.  Build the vocabulary.

In [124]:
def create_index(total_tweets):
        """
        Builds an index for the training and test set.
        The sets serve as a list of row numbers to extract from the dataset. 
        """
        # based on the total tweet count, create an array of all line numbers 
        line_index = np.array(range(0,total_tweets))
        # split the array into training and test sets of index values
        trainIndex,testIndex = train_test_split(line_index,train_size=0.70, random_state=42)
        # save test & traning index values
        #np.save("training_index",trainIndex)
        #np.save("testing_index",testIndex)
        return trainIndex,testIndex

trainIndex,testIndex = create_index(len(tweet_text))

In [125]:
# see http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

def vectorize_1(vocab=[]):
    vectorizer = TfidfVectorizer(#min_df=20
                                 stop_words='english'
                                 , use_idf=True # enable inverse-document-frequency reweighting
                                 , ngram_range=(1,2) # given our vocab, not really necessary
                                 , binary = True # presence of word instead of frequency
                                 #, vocabulary = vocab
                                ) 
    #X = vectorizer.fit_transform(tweet_list)
    return vectorizer

def vectorize_2():
    vectorizer = CountVectorizer(min_df=20
                                 , stop_words='english'
                                 , ngram_range=(1,2) # given our vocab, not really necessary
                                 , binary = True # presence of word instead of frequency
                                 #, vocabulary = set(vocab)
                                ) 
    #X = vectorizer.fit_transform(tweet_list)
    return vectorizer


In [129]:
vectorizer = vectorize_1()
X = vectorizer.fit_transform([tweet_text[i] for i in trainIndex])

In [ ]:
#explained_variances = np.var(X_svd, axis=0) / np.var(X_train, axis=0).sum()

### 
### This can take 10-15 mins
###
def create_svd_doc_term_matrix(X_train, num_eigen_vectors=100):
    """
    Create the array with truncated svd.
    """
    svd = TruncatedSVD(n_components = num_eigen_vectors)
    lsa = make_pipeline(svd, Normalizer(copy=False))
    return lsa.fit_transform(X_train), svd

run_here = False
if run_here:
    explained_variance_red = []
    explained_variance_nouns = []
    svd_component_range = range(100,351,50)
    for i in svd_component_range:
        # find explained variance in manual reduction method
        X_svd_red, svd_red = create_svd_doc_term_matrix(X_reduced,i)
        explained_variance_red.append(svd_red.explained_variance_ratio_.sum())
        # find explained variance in noun reduction method
        X_svd_noun, svd_noun = create_svd_doc_term_matrix(X_noun,i*3)
        explained_variance_nouns.append(svd_noun.explained_variance_ratio_.sum())

